# Plant Pathology 2020 - EfficientNetB0
https://www.kaggle.com/c/plant-pathology-2020-fgvc7

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

op = os.path.join

In [2]:
import torch 

### CUDA GPU Device Check

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Path

In [4]:
path = "/home/dmsai2/Desktop/AI-Study/PlantPathology2020/"
train_csv = op(path, "train.csv")
train_path = op(path, "images", "Train")

In [5]:
print("train:", len(os.listdir(train_path)))
n_train_data = len(os.listdir(train_path))

train: 1821


In [6]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.io import read_image

### Transform for DataLoader

In [7]:
from torchvision import transforms

In [8]:
tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

### PyTorch Customized `Datasets` Class
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [9]:
import torch.nn.functional as F

In [10]:
class PlantPathologyDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, header=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path + '.jpg')
        image = image.type(torch.FloatTensor)
        image = image / 255.
    
        label = np.argmax(self.img_labels.iloc[idx, 1:].values)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### Load Datasets

In [11]:
dataset = PlantPathologyDataset(train_csv, train_path, transform=tf)

### Show Image Example

In [12]:
def show_images(data, is_test=False):
    f, ax = plt.subplots(5, 5, figsize=(15, 10))
    
    for i in range(25):
        img_dir = data.img_labels.iloc[i, 0]
        img_data = cv2.imread(op(train_path, img_dir + '.jpg'))
        label = np.argmax(data.img_labels.iloc[0, 1:].values)
        
        if label  == 0:  str_label = 'healthy'
        elif label == 1:  str_label = 'multiple_diseases'
        elif label == 2: str_label = 'rust'
        else: str_label = 'scab'
        if(is_test): str_label="None"
        
        ax[i//5, i%5].imshow(img_data)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))
        
    plt.show()

In [13]:
# show_images(dataset)

### Train Test Dataset Split

In [14]:
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

In [15]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [16]:
print(f"Training Data Size : {len(train_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

Training Data Size : 1456
Testing Data Size : 365


### Load Dataset

In [17]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 16

In [18]:
train_dataloader = DataLoader(train_dataset, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True, 
                              drop_last=True)
test_dataloader = DataLoader(test_dataset,
                             batch_size=TEST_BATCH_SIZE, 
                             shuffle=True, 
                             drop_last=True)

In [19]:
X_train, y_train = next(iter(train_dataloader))
print(f"Feature batch shape: {X_train.size()}")
print(f"Labels batch shape: {y_train.size()}")

/root/anaconda3/envs/hackthon2/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Feature batch shape: torch.Size([64, 3, 224, 224])
Labels batch shape: torch.Size([64])


### Activation Function (Swish based ReLU)

In [20]:
def relu_fn(x):
    """ Swish activation function """
    return x * torch.sigmoid(x)

### EfficientNetB0 Model
https://startnow95.tistory.com/4 <br>
https://deep-learning-study.tistory.com/563

In [21]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Conv2dSamePadding(nn.Conv2d):
    """ 2D Convolutions like TensorFlow """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride, 0, dilation, groups, bias)

    def forward(self, x):
        ih, iw = x.size()[-2:]
        kh, kw = self.weight.size()[-2:]
        sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w//2, pad_w - pad_w//2, pad_h//2, pad_h - pad_h//2])
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

def drop_connect(inputs, p, training):
    """ Drop connect. """
    if not training: return inputs
    batch_size = inputs.shape[0]
    keep_prob = 1 - p
    random_tensor = keep_prob
    random_tensor += torch.rand([batch_size, 1, 1, 1], dtype=inputs.dtype)  # uniform [0,1)
    binary_tensor = torch.floor(random_tensor)
    output = inputs / keep_prob * binary_tensor
    return output

class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block
    """

    def __init__(self, kernel_size, stride, expand_ratio, input_filters, output_filters, se_ratio, drop_n_add):
        super().__init__()
        
        self._bn_mom = 0.1
        self._bn_eps = 1e-03
        self.has_se = (se_ratio is not None) and (0 < se_ratio <= 1)
        self.expand_ratio = expand_ratio
        self.drop_n_add = drop_n_add

        # Filter Expansion phase
        inp = input_filters  # number of input channels
        oup = input_filters * expand_ratio  # number of output channels
        if expand_ratio != 1: # add it except at first block 
            self._expand_conv = Conv2dSamePadding(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
            self._bn0 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Depthwise convolution phase
        k = kernel_size
        s = stride
        self._depthwise_conv = Conv2dSamePadding(
            in_channels=oup, out_channels=oup, groups=oup,  # groups makes it depthwise(conv filter by filter)
            kernel_size=k, stride=s, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Squeeze and Excitation layer, if desired
        if self.has_se:
            num_squeezed_channels = max(1,int(input_filters * se_ratio))  # input channel * 0.25 ex) block2 => 16 * 0.25 = 4
            self._se_reduce = Conv2dSamePadding(in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1)
            self._se_expand = Conv2dSamePadding(in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1)

        # Output phase
        final_oup = output_filters
        self._project_conv = Conv2dSamePadding(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
        self._bn2 = nn.BatchNorm2d(num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps)
        
    def forward(self, inputs, drop_connect_rate=0.2):

        # Expansion and Depthwise Convolution
        x = inputs.to(device)
        if self.expand_ratio != 1:
            x = relu_fn(self._bn0(self._expand_conv(inputs)))
        x = relu_fn(self._bn1(self._depthwise_conv(x)))

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_expand(relu_fn(self._se_reduce(x_squeezed)))
            x = torch.sigmoid(x_squeezed) * x
            
        # Output phase
        x = self._bn2(self._project_conv(x))

        # Skip connection and drop connect
        if self.drop_n_add == True:
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x

class EfficientNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Batch norm parameters
        bn_mom = 0.1
        bn_eps = 1e-03

        # stem
        in_channels = 3
        out_channels = 32
        self._conv_stem = Conv2dSamePadding(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Build blocks
        self._blocks = nn.ModuleList([]) # list 형태로 model 구성할 때
        # stage2 r1_k3_s11_e1_i32_o16_se0.25
        self._blocks.append(MBConvBlock(kernel_size=3, stride=1, expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, drop_n_add=False))
        # stage3 r2_k3_s22_e6_i16_o24_se0.25
        self._blocks.append(MBConvBlock(3, 2, 6, 16, 24, 0.25, False))
        self._blocks.append(MBConvBlock(3, 1, 6, 24, 24, 0.25, True))
        # stage4 r2_k5_s22_e6_i24_o40_se0.25
        self._blocks.append(MBConvBlock(5, 2, 6, 24, 40, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 40, 40, 0.25, True))
        # stage5 r3_k3_s22_e6_i40_o80_se0.25
        self._blocks.append(MBConvBlock(3, 2, 6, 40, 80, 0.25, False))
        self._blocks.append(MBConvBlock(3, 1, 6, 80, 80, 0.25, True))
        self._blocks.append(MBConvBlock(3, 1, 6, 80, 80, 0.25, True))
        # stage6 r3_k5_s11_e6_i80_o112_se0.25
        self._blocks.append(MBConvBlock(5, 1, 6, 80,  112, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 112, 112, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 112, 112, 0.25, True))
        # stage7 r4_k5_s22_e6_i112_o192_se0.25
        self._blocks.append(MBConvBlock(5, 2, 6, 112, 192, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        # stage8 r1_k3_s11_e6_i192_o320_se0.25
        self._blocks.append(MBConvBlock(3, 1, 6, 192, 320, 0.25, False))

        # Head 
        in_channels = 320
        out_channels = 1280
        self._conv_head = Conv2dSamePadding(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Final linear layer
        self._dropout = 0.2
        self._num_classes = 10
        self._fc = nn.Linear(out_channels, self._num_classes)

    def extract_features(self, inputs):
        """ Returns output of the final convolution layer """

        # Stem
        x = relu_fn(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):          
            x = block(x)
        return x

    def forward(self, inputs):
        """ Calls extract_features to extract features, applies final linear layer, and returns logits. """
        
        # Convolution layers
        x = self.extract_features(inputs).to(device)

        # Head
        x = relu_fn(self._bn1(self._conv_head(x)))
        x = F.adaptive_avg_pool2d(x, 1).squeeze(-1).squeeze(-1)
        if self._dropout:
            x = F.dropout(x, p=self._dropout, training=self.training)
        x = self._fc(x)
        return x

### Build Model

In [22]:
import torchsummary
from torchsummary import summary as summary_
import math

In [23]:
from efficientnet_pytorch import EfficientNet

In [25]:
model = EfficientNet.from_name('efficientnet-b0', 
                               num_classes=4,
                               dropout_rate=0.5).to(device)
summary_(model, (3, 224, 224), batch_size=2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1           [2, 3, 225, 225]               0
Conv2dStaticSamePadding-2          [2, 32, 112, 112]             864
       BatchNorm2d-3          [2, 32, 112, 112]              64
MemoryEfficientSwish-4          [2, 32, 112, 112]               0
         ZeroPad2d-5          [2, 32, 114, 114]               0
Conv2dStaticSamePadding-6          [2, 32, 112, 112]             288
       BatchNorm2d-7          [2, 32, 112, 112]              64
MemoryEfficientSwish-8          [2, 32, 112, 112]               0
          Identity-9              [2, 32, 1, 1]               0
Conv2dStaticSamePadding-10               [2, 8, 1, 1]             264
MemoryEfficientSwish-11               [2, 8, 1, 1]               0
         Identity-12               [2, 8, 1, 1]               0
Conv2dStaticSamePadding-13              [2, 32, 1, 1]             288
         I

### Neptune AI

In [26]:
import neptune

In [27]:
run = neptune.init_run(
    project="leehe228/plant-pathology",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5MTRmYjRlNC0zODFlLTQ0ODItODY1MC1hZGQ0YTRhNDNlZjIifQ==",
)

/tmp/ipykernel_2018668/590607257.py:1: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/leehe228/plant-pathology/e/P001-41


### Params

In [29]:
params = {
    # lr 1e-3 ~ 1e-6
    "learning_rate": 1e-4,
    "batch_size": BATCH_SIZE,
    "input_size": 3 * 244 * 244,
    "num_epoch": 25,
    "n_classes": 4,
    "optimizer": 'Adam',
    "criterion": 'CrossEntropyLoss',
    "preproc_type": "transform",
    "model":"EfficientNetB0",
    "library":"PyTorch",
    "dropout":0.5,
    "device": str(device)
}
# add parameters
run["parameters"] = params

### Optimizer and Criterion

In [30]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

### Metric: ROC AUC

In [31]:
from torcheval.metrics.aggregation.auc import AUC
from torchmetrics import AUROC

In [32]:
# metric_auc = AUC()
metric_auc = AUROC(task="multiclass", num_classes=4)

### Learning Scheduler

In [33]:
# T_max : number of iter
# eta_min : min value of learning rate
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, 
#                                                        T_max=5, 
#                                                        eta_min=1e-6,
#                                                        last_epoch=-1,
#                                                        verbose=True)

### Training Funcion

In [34]:
import gc

In [35]:
def train(model, params, criterion, optimizer):
    
    optim_lr = optimizer.param_groups[0]["lr"]
    run["train/epoch/lr"].append(optim_lr)
        
    for epoch in range(0, params['num_epoch']):
        print(f"epoch {epoch + 1}.")
        
        for i, data in enumerate(train_dataloader, 0):
            # data, label 분리
            x_train, y_train = data
            
            # y_train to one_hot_encoding
            labels = F.one_hot(y_train, num_classes=4).double()
        
            inputs = x_train.to(device)
            labels = labels.to(device)

            # 이전 batch에서 계산된 가중치 초기화
            optimizer.zero_grad()

            # forward + back propagation
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # calculate loss, acc, roc auc
            train_loss = criterion(outputs, labels)
            train_acc = (torch.sum(preds == torch.argmax(labels, dim=1))).sum().item() / len(inputs) 
            train_auc = metric_auc(labels, preds).item()
            
            print(f"batch {i}/{n_train_data//BATCH_SIZE} acc:{train_acc}, auc:{train_auc}, loss:{train_loss}")
            
            # training batch loss and accuracy, auc
            run["train/batch/loss"].append(train_loss)
            run["train/batch/acc"].append(train_acc)
            run['train/batch/auc'].append(train_auc)
            
            train_loss.backward()
            optimizer.step()
            metric_auc.reset()
        
        # empty GPU RAM
        torch.cuda.empty_cache()
        gc.collect()
        
        # test accuracy
        total = 0
        correct = 0
        accuracy = []
        auc = []
        losses = []
        
        for i, data in enumerate(test_dataloader, 0):
            x_train, y_train = data
            
            # y_train to one_hot_encoding
            labels = F.one_hot(y_train, num_classes=4).double()
        
            inputs = x_train.to(device)
            labels = labels.to(device)
            
            with torch.no_grad():
                outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (torch.sum(preds == torch.argmax(labels, dim=1))).sum().item()
            test_loss = criterion(outputs, labels).item()
            test_auc = (metric_auc(labels, preds)).item()
            test_accuracy = correct / total
        
            run["test/batch/loss"].append(test_loss)
            run["test/batch/acc"].append(test_accuracy)
            run['test/batch/auc'].append(test_auc)
            
            accuracy.append(test_accuracy)
            auc.append(test_auc)
            losses.append(test_loss)
            metric_auc.reset()
        
        # empty GPU RAM
        torch.cuda.empty_cache()
        gc.collect()
        
        num_epochs = params["num_epoch"]
        print(f"epoch: {epoch+1}/{num_epochs}, Test Loss: {np.mean(losses)}, Test Acc: {np.mean(accuracy)}, Test AUC: {np.mean(auc)}")
        print("\n")
        print("="*30)
        
        run["test/epoch/loss"].append(np.mean(losses))
        run["test/epoch/acc"].append(np.mean(accuracy))
        run['test/epoch/auc'].append(np.mean(auc))
        
        # scheduler
        # scheduler.step(np.mean(auc))
        
        optim_lr = optimizer.param_groups[0]["lr"]
        print(f"learning rate epoch {epoch} : {optim_lr}")
        run["train/epoch/lr"].append(optim_lr)

In [36]:
train(model, params, criterion, optimizer)

epoch 1.


/root/anaconda3/envs/hackthon2/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


batch 0/28 acc:0.21875, auc:0.4857335090637207, loss:1.4361334573477507
batch 1/28 acc:0.375, auc:0.5527283549308777, loss:1.405039700679481
batch 2/28 acc:0.265625, auc:0.4738917946815491, loss:1.435982882976532


/root/anaconda3/envs/hackthon2/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


batch 3/28 acc:0.4375, auc:0.4408894032239914, loss:1.2980095997918397
batch 4/28 acc:0.328125, auc:0.49870800971984863, loss:1.343501664698124
batch 5/28 acc:0.4375, auc:0.4614075720310211, loss:1.3263648296706378
batch 6/28 acc:0.40625, auc:0.41284826397895813, loss:1.3611409915611148
batch 7/28 acc:0.421875, auc:0.4185192584991455, loss:1.3090643286705017
batch 8/28 acc:0.34375, auc:0.4055669903755188, loss:1.3995352378115058
batch 9/28 acc:0.3125, auc:0.36064836382865906, loss:1.3924778664950281
batch 10/28 acc:0.40625, auc:0.39348389208316803, loss:1.3506021574139595
batch 11/28 acc:0.40625, auc:0.49998974800109863, loss:1.2725182776339352
batch 12/28 acc:0.5, auc:0.6238123178482056, loss:1.263876212760806
batch 13/28 acc:0.4375, auc:0.5898457765579224, loss:1.2849973663687706
batch 14/28 acc:0.390625, auc:0.4030410349369049, loss:1.3066226574592292
batch 15/28 acc:0.546875, auc:0.606717050075531, loss:1.2313218768686056
batch 16/28 acc:0.390625, auc:0.41529643535614014, loss:1.29

In [37]:
model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

### Save Model State 

In [38]:
import torchvision.models as models

In [39]:
torch.save(model.state_dict(), './weights/model_weight_efficientB0_230513(2).pt')

### Submission

In [40]:
pred_list = []
pred_list.append(['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab'])

In [41]:
submission_data = pd.read_csv('test.csv')
submission_data.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [42]:
submission_path = '/home/dmsai2/Desktop/AI-Study/PlantPathology2020/images/Submission'

In [43]:
from tqdm import tqdm

In [44]:
for i in tqdm(range(len(submission_data))):
    img_title = submission_data.iloc[i, 0]
    img_dir = img_title + '.jpg'
    # print(op(submission_path, img_dir))
    
    image = read_image(op(submission_path, img_dir))
    image = image.type(torch.FloatTensor)
    img = image / 255.
    img = tf(img)
    img = img.reshape(1, 3, 224, 224)
    inputs = img.to(device)
    # print(inputs.shape)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    ans = [img_title, 0, 0, 0, 0]
    ans[int(preds) + 1] = 1
    pred_list.append(ans)
print(len(pred_list))

  0%|                                                  | 0/1821 [00:00<?, ?it/s]/root/anaconda3/envs/hackthon2/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|███████████████████████████████████████| 1821/1821 [01:21<00:00, 22.30it/s]

1822


In [45]:
submission_df = pd.DataFrame(pred_list)
submission_df.head()

,0,1,2,3,4
0,image_id,healthy,multiple_diseases,rust,scab
1,Test_0,0,0,1,0
2,Test_1,1,0,0,0
3,Test_2,0,0,0,1
4,Test_3,0,0,1,0


In [46]:
submission_df.to_csv('./submission/submission_efnet2.csv', index=False)